- #### Arquitectura
- #### Flujo de ejecución
- #### Transformaciónes vs Acciones
- #### Optimizador Catalist
- #### DAG

## Spark architecture

In [1]:
from IPython.display import display, Image
display(Image(url='../resources/img/SPARK.png'))

#### Driver program
The process running the main() function of the application and creating the SparkContext. **Starts the application and sends work to the executors.**

#### Cluster manager
An external service for acquiring resources on the cluster (e.g. standalone manager, Mesos, YARN, Kubernetes)

   - **Standalone** – a simple cluster manager included with Spark that makes it easy to set up a cluster.
   - **Apache Mesos** – a general cluster manager that can also run Hadoop MapReduce and service applications. (Deprecated)
   - **Hadoop YARN** – the resource manager in Hadoop 2 and 3.
   - **Kubernetes** – an open-source system for automating deployment, scaling, and management of containerized applications.

#### Worker node	
Any node that can run application code in the cluster

#### Executor
A process launched for an application on a worker node, that runs tasks and keeps data in memory or disk storage across them. Each application has its own executors. **Launched in JVM containers with their own memory and CPU resources**

#### Deploy mode	
Distinguishes where the driver process runs. 

- **Cluster mode** - the Spark driver is launched on a worker node
- **Client mode** - the Spark driver is on the client machine
- **Local mode** - the entire application runs on the same machine.

## Spark app execution

- An action triggers a **Job**
  
- A job is split into **stages**
    - each stage is dependent on the stage before it
    - a stage must fully complete before the next stage can start
    - for performance (usually) minimize the number of stages

- A stage has **tasks**
    - task = smallest unit of work
    - tasks are run by executors

- **An RDD/DF/DS has partitions**

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("sesion_1") \
        .master("local[*]") \
        .getOrCreate()

sc = spark.sparkContext

In [3]:
## Stage 1
employees = sc.textFile("../resources/data/csv/employees.csv")
empTokens = employees.map(lambda line: line.split(","))
empDetails = empTokens.map(lambda tokens: (tokens[4], tokens[7]))
## Stage 2
empGroups = empDetails.groupByKey(2)
avgSalaries = empGroups.mapValues(lambda salaries: sum([int(item) for item in salaries]) / len(salaries))

In [4]:
avgSalaries

PythonRDD[6] at RDD at PythonRDD.scala:53

#### **TRANSFORMATIONS ARE LAZY - SO ARE EXECUTED WHEN AN ACTION IS TRIGGERED**

In [5]:
avgSalaries.foreach(print)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 0.0 failed 1 times, most recent failure: Lost task 1.0 in stage 0.0 (TID 1) (23LAP executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:192)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:166)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:328)
	at org.apache.spark.api.python.PairwiseRDD.compute(PythonRDD.scala:128)
	at org.apache.spark.rdd

In [ ]:
display(Image(url='../resources/img/Stages_tasks.PNG'))

#### Shuffle

- Exchange of data between executors
- happens in between stages
- must complete before next stage starts

- Expensive because of
    - transferring data
    - serialization/deserialization
    - loading new data from shufflefiles
- Shuffles are performance bottlenecks because
    - exchanging data takes time
    - they need to be fully completed before next computations start
- Shuffles limit parallelization

  

#### Concepts Relationship
App decomposition
- 1 job = 1 or more stages
- 1 stage = 1 or more tasks

Tasks & executors
- 1 task is run by 1 executor
- each executor can run 0 or more tasks

Partitions and tasks
- processing 1 partition = one task

Partitions & executors
- 1 partition stays on 1 executor
- each executor can load 0 or more partitions in memory or disk

Executors & nodes
- 1 executor = 1 JVM on 1 physical node
- each physical node can have 0 or more executors

## Transformations vs Actions

- Transformations describe how new DFs are obtained
- Actions start executing Spark code
- Transformations return RDDs/DFs
- Actions return something else e.g. Unit, number, etc.

#### Narrow Transformations

- given a parent partition, a single child partition depends on it
- fast to compute
- examples: union, map, flatMap, select, filter

#### Wide transformations

- given a parent partition, more than one child partitions depend on it
- involve a shuffle = data transfer between Spark executors
- are costly to compute
- examples: joining, grouping, sorting

## Catalyst Optimizer

In [ ]:
display(Image(url='../resources/img/Catalist_Optimizer.png'))

1. Catalyst Optimizer and Tungsten Execution Engine was introduced in Spark 1.x
2. Cost-Based Optimizer was introduced in Spark 2.x
3. Adaptive Query Execution now got introduced in Spark 3.x

    - To disable the Adaptive Query Execution -> spark.conf.set("spark.sql.adaptive.enabled", False)

Only works with DF and DS

In [ ]:
ds1 = spark.range(1, 100000000)
ds2 = spark.range(1, 100000000, 2)
ds3 = ds1.repartition(7)
ds4 = ds2.repartition(9)
ds5 = ds3.selectExpr("id * 3 as id")
joined = ds5.join(ds4, "id")
sum = joined.selectExpr("sum(id)")
sum.explain()

## DAG

DAG = Directed Acyclic Graph = visual representation of Spark jobs


In [ ]:
spark

In [ ]:
sum.count() # Call an action to trigger transformations

In [ ]:
display(Image(url='../resources/img/DAG.png'))